In [1]:
import pandas as pd
import pyodbc 
from openpyxl import load_workbook
from datetime import datetime

print("Connection Start")

server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""
CoverageQuery = """
SELECT
[UserName] AS 'Emp User ID'
,[FullName] AS 'Employee Name'
,[LISStoreCode] AS 'StoreCode'
,[StoreName] AS 'StoreName'
,[VisitDate] AS 'Visit Date'
,[CheckIn-Time] AS 'StartDateTime'
,[CheckOut-Time] AS 'EndDateTime'
,[TimeDuration] AS 'NoOfWorkingHrs'
,[NoPermissionReason] AS 'No Permission Reason'
, [PJPStatus] AS 'PJP Type'
FROM [PowerBI_LISV2].[DaburPharmaManagement].[ISPStoreAttendance_NormalExport]
where MONTH = 08 and [UserName] not like '%ISPStar%' AND [UserName] not like '%SUP%' and [UserName] not like  '%TEST%' 
ORDER BY [VisitDate] DESC;
-- FORMAT(TRY_CONVERT(date, A.[Date], 105), 'dd-MM-yyyy')

"""

PrimaryShelf = """
SELECT [VisitID]
	  ,[SubmittedDate&Time] AS 'Submitted Date&Time'
      ,[SyncDate&Time] as 'Sync Date&Time'
      ,[Month]
      ,[Year]
      ,[Region]
      ,[State]
      ,[City]
	  ,[RSMCode]
      ,[BranchCode]
      ,[ASMCode]
      ,[SOCode]
      ,[ISPCode]
      ,[Username]
      ,[FullName] as 'Full Name'
      ,[Designation]
      ,[ClientStoreCode] as 'Client Store Code'
      ,[LISStoreCode] as 'LIS Store Code'
      ,[StoreName] as 'Store Name'
      ,[StoreAddress] as 'Store Address'
      ,[Format]
      ,[ChainCode] as 'Chain Code'
      ,[ChainName] as 'Chain Name'
      ,[SubChainName] as 'Sub Chain Name'
      ,[StoreType] as 'Store Type'
      ,[StoreClub] as 'Store Club'
      ,[Distributorcode] as 'Distributor code'
      ,[DistributorName] as 'Distributor Name'
      ,[PJPStatus] as 'PJP Status'
      ,[ProductCode] as 'Product Code'
      ,[ProductName] as 'Product Name'
      ,[MRP]
      ,[MAQ]
      ,[ProductStatus] as 'Product Status'
      ,[MandatoryStatus] as 'Mandatory Status'
      ,[MSLStatus] as 'MSL Status'
      ,[Quantity]
      ,[Amount]
      ,[CSP]
	  ,[MFD]
	  ,[DamageQuantity] as 'Damage Quantity'
	  ,[MasterLatitude] as 'Master Latitude'
      ,[MasterLongitude] as 'Master Longitude'
      ,[Latitude] AS 'Punched Latitude'
      ,[Longitude] AS 'Punched Longitude'
      ,[Distance]
      ,[Comment]
  FROM [PowerBI_LISV2].[DaburPharmaManagement].[PrimaryShelfStockReportExport]
  where MONTH = 08 and [UserName] not like '%ISPStar%' AND [UserName] not like '%SUP%' and LOWER([UserName]) not like  '%test%' 

"""

PaidVisibility_DataQuery =""" SELECT [VisitID] AS [Visit ID]
	,[SubmittedDateandTime] AS 'Submitted Date and Time'
	,[SyncDateandTime] AS [Sync Date and Time]
	,[Month] 
	,[Year]
	,[Region]
	,[State]
	,[City]
	,[RSMCode]
	,[BranchCode]
	,[ASMCode]
	,[SOCode]
	,[SupervisorCode]
	,[ISPCode]
	,[Username]
	,[FullName] AS [Full Name]
	,[Designation]
	,[ClientStoreCode] AS [Client Store Code]
	,[LISStoreCode] AS [LIS Store Code]
	,[StoreName] AS [Store Name]
	,[StoreAddress] AS [Store Address]
	,[Format]
	,[ChainCode] AS [Chain Code]
	,[ChainName] AS [Chain Name]
	,[SubChainName] AS [Sub Chain Name]
	,[StoreType] AS [Store Type]
	,[StoreClub] AS [Store Club]
	,[DistributorCode] AS [Distributor Code]
	,[DistributorName] AS [Distributor Name]
	,[PJPStatus] AS [PJP Status]
	,[POSMCode] AS [POSM Code]
	,[POSMName] AS [POSM Name]
	,[POSMBrand] AS [POSM Brand]
	,[StartDate] AS [Start Date]
	,[EndDate] AS [End Date]
	,[AuditStatus] AS [AuditStatus]
	,[VisibilityAvailable] AS [Visibility Available]
	,[NonAvailabilityReason] AS [Non Availability Reason]
	,[CloseUpPhotoLink] AS [Close Up Photo Link]
	,[LeftSidePhotoLink] AS [Left Side Photo Link]
	,[RightSidePhotoLink] AS [Right Side Photo Link]
	,[Scanner1DetectedText] AS [Scanner 1 Detected Text]
	,[Scanner2DetectedText] AS [Scanner 2 Detected Text]
	,[Scanner3DetectedText] AS [Scanner 3 Detected Text]
	,[Scanner4DetectedText] AS [Scanner 4 Detected Text]
	,[Scanner5DetectedText] AS [Scanner 5 Detected Text]
	,[NonDetectionReason] AS [Non Detection Reason]
	,[AppQuestions] AS [App Questions]
	,[AppResponse] AS [App Response]
	,[PortalQuestions] AS [Portal Questions]
	,[PortalResponse] AS [Portal Response]
	,[PortalResponsePoints] AS [Portal Response Points]
  FROM [PowerBI_LISV2].[DaburPharmaManagement].[PaidVisibilityReportExport]
  where MONTH = 08 and [UserName] not like '%ISPStar%'and  [Username] not like '%SUP%' and LOWER([Username]) not like '%test%'

"""
conn = pyodbc.connect(conn_str)
Coverage = pd.read_sql(CoverageQuery, conn)
PrimaryShelf = pd.read_sql(PrimaryShelf, conn)

PaidVisibility_Data = pd.read_sql(PaidVisibility_DataQuery, conn)
Coverage.at[0, 'LastUpdated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
PrimaryShelf.at[0, 'LastUpdated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
PaidVisibility_Data.at[0, 'LastUpdated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

PromotionQuery = """
SELECT [VisitId] AS [Visit Id]
	,[SubmittedDate&Time] AS 'Submitted Date & Time'
	,[SyncDate&Time] AS [Sync Date & Time]
	,[Month] 
	,[Year] 
	,[Region]
	,[State]
	,[City]
	,[RSMCode]
	,[BranchCode]
	,[ASMCode]
	,[SOCode]
	,[SupervisorCode]
	,[ISPCode]
	,[Username]
	,[FullName] AS [Full Name]
	,[Designation] AS [Designation]
	,[ClientStoreCode] AS [Client Store Code]
	,[LISStoreCode] AS [LIS Store Code]
	,[StoreName] AS [Store Name]
	,[StoreAddress] AS [Store Address]
	,[Format]
	,[ChainCode] AS [Chain Code]
	,[ChainName] AS [Chain Name]
	,[SubChainName] AS [Sub Chain Name]
	,[StoreType] AS [Store Type]
	,[StoreClub] AS [Store Club]
	,[DistributorCode] AS [Distributor Code]
	,[DistributorName] AS [Distributor Name]
	,[PJPStatus] AS [PJP Status]
	,[ProductCategoryName] AS [Product Category Name]
	,[ProductBrandName] AS [Product Brand Name]
	,[ProductCode] AS [Product Code]
	,[ProductName] AS [Product Name]
	,[Offer]
	,[IsSamePromotionRunning] AS [Is Same Promotion Running]
	,[SameonPOS] AS [Same on POS]
	,[SameonShelftalker] AS [Same on Shelf talker]
	,[CountOfFacing] AS [Count Of Facing]
	,[Reason]
	,[PromotionType] AS [Promotion Type]
	,[PromotionComment] AS [Promotion Comment]
	,[PhotoLink] AS [Photo Link]
	,[Latitude] AS [Latitude]
	,[Longitude] AS [Longitude]
	,[AIDetectionStatus] AS [AI Detection Status]
	,[DetectionReason] AS [Detection Reason]
	,[SystemOffer] AS [System Offer]
	,[SystemFacings] AS [System Facings]
	,[AnnotatedImagePath] AS [Annotated Image Path]
	,[StockCount] AS [StockCount]
	,[MasterLatitude] AS [Master Latitude]
	,[MasterLongitude] AS [Master Longitude]
	,[PunchedLatitude] AS [Punched Latitude]
	,[PunchedLongitude] AS [Punched Longitude]
	,[Distance] AS [Distance]    
  FROM [PowerBI_LISV2].[DaburPharmaManagement].[PromotionTrackingReportExport]
  where MONTH = 08 and [UserName] not like '%ISPStar%'and  [Username] not like '%SUP%' and LOWER([Username]) not like '%test%'

"""


WindowVisibility = """
SELECT [VisibilityAuditInputId]
      ,[VisitId]
	  ,[SubmittedDate&Time] as 'Date'
      ,[SyncDate&Time]
      ,[Month]
      ,[Year]
      ,[Region]
      ,[State]
      ,[City]
      ,[Username]
      ,[FullName]
      ,[Designation]
      ,[ClientStoreCode]
      ,[LISStoreCode]
      ,[StoreName]
      ,[Format]
      ,[ChainCode]
      ,[ChainName]
      ,[POSMCode]
      ,[POSMName]
      ,[POSMBrand]
      ,[POSMStartDate]
      ,[POSMEndDate]
      ,[AuditStatus]
      ,[VisibilityAvailable]
      ,[PrePhotoLink]
      ,[PostCloseUpPhotoLink]
      ,[PostDistancePhotoLink]
      ,[PortalQuestions]
      ,[PortalResponse]
      ,[PortalResponsePoints]
      ,CAST([AuditedDate] As DATE)
  FROM [PowerBI_LISV2].[DaburPharmaManagement].[VisibilityAuditInputReport]
  where [Month] = 8 and [username] not like '%SUP%' and LOWER([username]) not like '%test%' and [PortalResponse] is not null and [AuditStatus]  = 'Audit'
"""

Promotion_Data = pd.read_sql(PromotionQuery, conn)
WindowVisibility= pd.read_sql(WindowVisibility, conn)

print("Query Executed")

Coverage.to_csv(r'C:\Users\Administrator\LIS\Dabur\Coverage.csv', index=False)
PrimaryShelf.to_csv(r'C:\Users\Administrator\LIS\Dabur\PrimaryShelf.csv', index=False)
PaidVisibility_Data.to_csv(r'C:\Users\Administrator\LIS\Dabur\PaidVisibility_Data.csv', index=False)
Promotion_Data.to_csv(r'C:\Users\Administrator\LIS\Dabur\Promotion_Data.csv', index=False)
WindowVisibility.to_csv(r'C:\Users\Administrator\LIS\Dabur\WindowVisibility.csv', index=False)

csv_files = [
    r'C:\Users\Administrator\LIS\Dabur\Coverage.csv',
    r'C:\Users\Administrator\LIS\Dabur\PrimaryShelf.csv',
    r'C:\Users\Administrator\LIS\Dabur\PaidVisibility_Data.csv',
    r'C:\Users\Administrator\LIS\Dabur\Promotion_Data.csv',
    r'C:\Users\Administrator\LIS\Dabur\WindowVisibility.csv'
]

print("Email Trigger Start")

# emailer

from datetime import datetime, timedelta
import smtplib
from email.message import EmailMessage
import mimetypes
import os
import zipfile


sender_email   = "reports.mis@loveinstore.com"
app_password   = "ydfd ntvg qcfp xxbi"
recipients     = ["varunj@loveinstore.com"]

# "kunalk@loveinstore.com","anandk@loveinstore.com",'pankajs@loveinstore.com']

subject_tpl    = "CSV Dump - {date} {time}"
body_template  = """
Please find attached Data for {date} at {time}

Regards,
Team Love In Store
"""


now = datetime.now() - timedelta(days=1)
rounded = now.replace(minute=0, second=0, microsecond=0)
time_str = rounded.strftime("%I:%M %p")
formatted_date = now.strftime("%d-%b-%Y")

# Create a ZIP archive
zip_file = r"C:\Users\Administrator\LIS\Dabur\Reports_{date}.zip".format(date=formatted_date.replace("-", ""))
with zipfile.ZipFile(zip_file, "w", zipfile.ZIP_DEFLATED) as zipf:
    for file_path in csv_files:
        zipf.write(file_path, os.path.basename(file_path))  # save only filename in ZIP
# Create email
msg = EmailMessage()
msg["From"]    = sender_email
msg["To"]      = ", ".join(recipients)
msg["Subject"] = subject_tpl.format(date=formatted_date, time=time_str)
msg.set_content(body_template.format(date=formatted_date, time=time_str))

# Attach ZIP
with open(zip_file, "rb") as f:
    file_data = f.read()
msg.add_attachment(file_data, maintype="application", subtype="zip", filename=os.path.basename(zip_file))

# Send email
with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
    smtp.login(sender_email, app_password)
    smtp.send_message(msg)

print(f"✅ Email sent with compressed attachment: {os.path.basename(zip_file)}")





Connection Start


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_26764\4194051254.py:151: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Coverage = pd.read_sql(CoverageQuery, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_26764\4194051254.py:152: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  PrimaryShelf = pd.read_sql(PrimaryShelf, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_26764\4194051254.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  PaidVisibility_Data = pd.read_sql(PaidVisibility_DataQuery, con

Query Executed
Email Trigger Start
✅ Email sent with compressed attachment: Reports_16Sep2025.zip
